Imports

In [27]:
import numpy as np
import pandas as pd

Loading Data

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
accounts = pd.read_parquet("/content/drive/MyDrive/Data/lgd_accounts.parquet")
accounts.head()

,customer_id,account_id,product_type,credit_limit,status_final,default_dt,balance_at_default
0,C01310,A00001,CREDIT_CARD,1000,CHARGED_OFF,2023-07-18,837.86
1,C00341,A00002,CREDIT_CARD,8000,ACTIVE,NaT,0.00
2,C00474,A00003,CREDIT_CARD,5000,ACTIVE,NaT,0.00
3,C01699,A00004,CREDIT_CARD,10000,CLOSED,NaT,0.00
4,C00895,A00005,CREDIT_CARD,2000,CLOSED,NaT,0.00


In [30]:
transactions = pd.read_parquet("/content/drive/MyDrive/Data/lgd_transactions.parquet")
transactions.head()

,customer_id,account_id,trx_id,trx_dt,trx_amount,trx_type,trx_message,post_balance
0,C01310,A00001,T0000001,2022-01-01,127.37,PURCHASE,PURCHASE_AT_MERCHANT_MISC,127.37
1,C01310,A00001,T0000002,2022-01-02,10.16,PURCHASE,PURCHASE_AT_MERCHANT_RESTAURANT,137.53
2,C01310,A00001,T0000003,2022-01-03,316.45,PURCHASE,PURCHASE_AT_MERCHANT_MISC,453.98
3,C01310,A00001,T0000004,2022-01-04,-144.85,PAYMENT,ONLINE_CARD_PAYMENT,309.13
4,C01310,A00001,T0000005,2022-01-04,-158.24,PAYMENT,CARD_PAYMENT,150.89


Cleaning

In [31]:
accounts["default_dt"] = pd.to_datetime(accounts["default_dt"])
transactions["trx_dt"] = pd.to_datetime(transactions["trx_dt"])


In [32]:
defaulted_accounts = accounts[accounts["default_dt"].notna()].copy()


#Part 3 Exposure at Default

In [33]:
defaulted_accounts["EAD"] = defaulted_accounts["balance_at_default"]


#Part 4 LGD

In [34]:
#Merging transactions and accounts

In [35]:
trx_merged = transactions.merge(
    defaulted_accounts[["account_id", "default_dt"]],
    on="account_id",
    how="inner"
)


#Recovery


In [36]:
#Filtering for recovery
recovery_trx = trx_merged[
    (trx_merged["trx_type"] == "PAYMENT") &
    (trx_merged["trx_dt"] > trx_merged["default_dt"])
].copy()


In [37]:
recovery_by_account = (
    recovery_trx
    .groupby("account_id")["trx_amount"]
    .sum()
    .abs()
    .reset_index()
    .rename(columns={"trx_amount": "recovery_amount"})
)


In [38]:
final_df = defaulted_accounts.merge(
    recovery_by_account,
    on="account_id",
    how="left"
)

final_df["recovery_amount"] = final_df["recovery_amount"].fillna(0)


In [39]:
final_df["LGD"] = 1 - (
    np.minimum(final_df["recovery_amount"], final_df["EAD"])
    / final_df["EAD"]
)


#Deliverabels

In [40]:
final_output = final_df[
    ["account_id", "EAD", "recovery_amount", "LGD"]
]

final_output.head()


,account_id,EAD,recovery_amount,LGD
0,A00001,837.86,833.32,0.005419
1,A00007,1750.32,1476.10,0.156668
2,A00008,1707.51,1653.43,0.031672
3,A00010,9901.44,9186.73,0.072182
4,A00012,9904.95,9904.49,0.000046


#Statistical Summaries

In [41]:
ead_summary = final_output["EAD"].describe(
    percentiles=[0.25, 0.5, 0.75]
)
ead_summary



,EAD
count,3417.000000
mean,4200.562022
std,3253.637935
min,7.570000
25%,1444.340000
50%,2900.630000
75%,7781.960000
max,10229.080000


In [42]:
lgd_summary = final_output["LGD"].describe(
    percentiles=[0.25, 0.5, 0.75]
)
lgd_summary

,LGD
count,3417.000000
mean,0.050028
std,0.134230
min,0.000000
25%,0.000583
50%,0.002724
75%,0.030542
max,1.000000


#Exporting to Excel

In [50]:
output_path = "LGD_EAD_results.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    final_output.to_excel(writer, sheet_name="Account_Level_Results", index=False)
    ead_summary.to_excel(writer, sheet_name="EAD_Summary", index=False)
    lgd_summary.to_excel(writer, sheet_name="LGD_Summary", index=False)


In [51]:
from google.colab import files

files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>